In [130]:
from pathlib import Path

import pandas as pd
import numpy as np

## Categories
1. Housing

- Rent/Mortgage
- Property Taxes
- Home Insurance
- Maintenance/Repairs
- Utilities (Electricity, Water, Gas)

2. Transportation
- Car Payments/Leasing
- Fuel
- Insurance
- Public Transportation
- Maintenance and Repairs
- Parking Fees/Tolls

3. Food & Groceries
- Groceries
- Dining Out
- Coffee/Snacks
- Meal Delivery Services

4. Health & Wellness

Health Insurance
Medical Bills (Doctor, Dentist, etc.)
Medications
Fitness (Gym Memberships, Classes)
Personal Care (Haircuts, Skincare)

5. Entertainment & Leisure

Streaming Subscriptions
Movies, Concerts, Events
Books, Magazines
Hobbies (Music, Crafts, etc.)
Travel/Vacations

6. Personal & Family Care

Childcare/Schooling
Pet Care (Food, Vet)
Clothing & Accessories
Gifts & Special Occasions

7. Insurance

Life Insurance
Disability Insurance
Home/Renters Insurance
Vehicle Insurance

8. Debt Repayment

Credit Card Payments
Student Loans
Personal Loans
Mortgage

9. Savings & Investments

Emergency Fund
Retirement Contributions
Stocks/Mutual Funds
Savings Accounts

10. Miscellaneous

Donations/Charity
Professional Services (Legal, Accounting)
Miscellaneous Fees/Bank Fees
Unexpected/One-time Expenses

In [131]:
DATA_DIR = Path('data')
FNAME = list(DATA_DIR.glob('*.csv'))[0]

In [182]:
header = ['Date', 'Exe Date', 'Title', 'Vendor', 'Account', 'Amount', 'Balance', '']
df = pd.read_csv(FNAME, skiprows=1, names=header, index_col=7, usecols=range(8))
df.Vendor = df.Vendor.astype(str)
df.Amount = df.Amount.str.replace(',', '.').astype(float) * -1
df.Balance = df.Balance.str.replace(',', '.').astype(float)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
DATA = df.round(2)

In [133]:
pd.options.display.max_rows = 200
pd.set_option('display.max_colwidth', None)

In [242]:
CATEGORIES = {
  'income'        : ['citibank', 'mishchenko'],
  'lokaty'        : ['lokaty'],
  'blik'          : ['blik'],
  'constr'        : ['leroy-merlin', 'market obi'],
  'groc'          : ['biedronka', 'lidl', 'aldi', 'zabka', 'spolem', 'kaufland', 'netto', 'delikatesy', 'kiosk', 'sklepik', 'lewiatan', 'food', 'transgourmet'],
  'drug'          : ['rossmann', 'dm-drog', 'apteka', 'dom lek'],
  'bakery'        : ['piekarnia', 'familijna', 'cukiernia', 'rozana', 'coffee', 'rogalik', 'cafe', 'puri', 'tandyr', 'paczkarnia', 'precel', 'helen', 'kawiarnia'],
  'ice_cream'     : ['lodziarnia', 'ice sweet', 'roma', 'agawa', 'lody'],
  'restaurants'   : ['pedet', 'padet', 'ekushe', 'mcdonalds', 'chinski', 'gordito', 'pod papu', 'aston', 'restauracja', 'kebab', 'kocur', 'pizza', 'ramen', 'pasibus', 'sevi',
                     'pierogarnia', 'gastro', 'burger', 'pinta'],
  'clothing'      : ['deichmann', 'sinsay'],
  'car'           : ['rentalcars', 'uber', 'bolt', 'orlen', 'bp-lena', 'paliw', 'parking', 'circle k', 'carsharing'],
  'train'         : ['koleje', 'pkp'],
  'pt'            : ['urbancard'],
  'consumer_goods': ['amazon', 'allegro', 'olx', 'free look', 'duka', 'a368', 'pepco', 'ikea'],
  'hotel'         : ['hotel', 'resort', 'altus', 'reso'],
  'culture'       : ['teatr', 'forum muzyki', 'zoo', 'museum', 'muzeum'],
  'atm'           : ['bankomat'],
  'igi'           : ['lego', 'smyk', 'park wodn'],
  'personal_care' : ['piekna', 'fryzjer', 'hair'],
}
ALL = sum(CATEGORIES.values(), start=['blik'])


In [274]:
df = DATA.copy()
a = pd.DataFrame(index=pd.MultiIndex.from_tuples([(y, m) for y in [2023, 2024] for m in range(1, 13)] + [('SUM', '')], names=['Year', 'Month']))
for cat, vals in CATEGORIES.items():
  col = ['Title', 'Vendor'][cat in ['income']]
  cut = df[col].str.lower().str.contains('|'.join(vals))
  df_ = df[cut]
  a[cat.upper()] = df_.groupby([df_['Date'].dt.year, df_['Date'].dt.month]).Amount.sum()
  df = df[~cut]  # remove to avoid double counting
a['OTHER'] = df.groupby([df['Date'].dt.year, df['Date'].dt.month]).Amount.sum()
a.loc[('SUM', '')] = a.sum()
a = a.fillna(0).iloc[3:]
a

INCOME    LOKATY      BLIK   CONSTR      GROC     DRUG  \
Year Month                                                              
2023 4       -1650.00      0.00    545.48     0.00   1475.24     7.99   
     5      -10900.90      0.00    685.40    94.38   1017.67   192.21   
     6       -5000.00      0.00    793.04   243.15   1677.97   187.62   
     7       -9750.00      0.00   5177.89   338.82   1354.97   209.96   
     8       -8000.00      0.00    172.40    64.72   1384.13   160.56   
     9       -7500.00      0.00   2820.39   291.57   1261.38   197.45   
     10     -10430.00   1030.00   3338.34     0.00    809.44   447.49   
     11      -6830.00   1830.00   2097.15   163.17   1147.82   364.69   
     12      -6830.00   1830.00    482.04     0.00   1093.91   544.22   
2024 1       -9574.00   1830.00   6335.05    57.75   1225.43   321.03   
     2       -6830.00   1830.00   1671.45     0.00   1260.83   421.51   
     3       -6670.00   1830.00   2631.70     0.00   1636.32   230.60   
     4       -6830.00   1830.00   1652.98   403.94   1308.96   154.58   
     5      -10966.00  -3704.38   2438.62    54.95    751.09   177.69   
     6       -8168.28   1830.00   1649.46     0.00    887.60   342.26   
     7       -7881.00   1830.00   1507.08     0.00    862.03   278.63   
     8       -4830.00      0.00   1269.62     0.00   1177.94   224.19   
     9       -8850.00   1830.00    606.15     0.00   1078.41   246.91   
     10      -6200.00     -0.01   1352.53   659.00   1078.82   232.05   
     11          0.00      0.00      0.00     0.00      0.00     0.00   
     12          0.00      0.00      0.00     0.00      0.00     0.00   
SUM        -143690.18  13795.61  37226.77  2371.45  22489.96  4941.64   

             BAKERY  ICE_CREAM  RESTAURANTS  CLOTHING      CAR  TRAIN     PT  \
Year Month                                                                     
2023 4       146.76       0.00        75.70      0.00    57.45    0.0   60.0   
     5        77.64       0.00       309.85      0.00   477.36    0.0  134.8   
     6       185.56      80.00        39.00      0.00    20.55  260.8  234.2   
     7       238.27      58.00       162.48     19.98    43.04    0.0  283.6   
     8       144.09      29.00       168.50      0.00     0.00  124.0   89.2   
     9       101.82      45.00        53.00      0.00    53.26    0.0   29.2   
     10      217.24       7.00       355.59    118.97     0.00    0.0    6.4   
     11       73.07      55.00       148.50      0.00     0.00    0.0    0.0   
     12      100.71       0.00       172.90      0.00     0.00    0.0   12.8   
2024 1         0.00       0.00         0.00      0.00   366.58    0.0    8.0   
     2       112.69       0.00        70.40      0.00     0.00    0.0    0.0   
     3       192.69       0.00         0.00      0.00     0.00    0.0   14.4   
     4       275.18       0.00         0.00      0.00     0.00    0.0    0.0   
     5       162.74      36.00       231.20    104.99     0.00    0.0   26.8   
     6        14.49      59.00       187.00      0.00   193.90    0.0   46.4   
     7       142.04      41.31       164.20      0.00     0.00    0.0   16.0   
     8        32.07      90.56        36.00      0.00  1027.63    0.0   12.4   
     9        83.22     140.31       111.20      0.00     0.00    0.0    0.0   
     10      236.23      20.00       307.90    159.99    13.00    0.0    0.0   
     11        0.00       0.00         0.00      0.00     0.00    0.0    0.0   
     12        0.00       0.00         0.00      0.00     0.00    0.0    0.0   
SUM         2536.51     661.18      2593.42    403.93  2252.77  384.8  974.2   

            CONSUMER_GOODS    HOTEL  CULTURE     ATM     IGI  PERSONAL_CARE  \
Year Month                                                                    
2023 4               53.00     0.00      0.0     0.0    0.00            0.0   
     5              610.93     0.00      0.0     0.0    0.00           40.0   
     6                0.0

In [271]:
a.index

MultiIndex([(2023,  4),
            (2023,  5),
            (2023,  6),
            (2023,  7),
            (2023,  8),
            (2023,  9),
            (2023, 10),
            (2023, 11),
            (2023, 12),
            (2024,  1),
            (2024,  2),
            (2024,  3),
            (2024,  4),
            (2024,  5),
            (2024,  6),
            (2024,  7),
            (2024,  8),
            (2024,  9),
            (2024, 10),
            (2024, 11),
            (2024, 12)],
           names=['Year', 'Month'])

In [241]:
df = df.copy()
c0, c1, c2, c3, c4 = np.ones((5, len(df)))
# c0 = df.Title.str.lower().str.contains('kolej')
# c0 = df.Title.str.lower().str.contains('|'.join(CATEGORIES['groc']))
# c0 = ~df.Title.str.lower().str.contains('|'.join(ALL))
# c1 = df.Vendor == 'nan'
c1 = df.Date.dt.month == 1
c2 = df.Date.dt.year == 2024
# c3 = df.Amount > 0
c = np.all([c0, c1, c2, c3, c4], axis=0)
# df[c][['Date', 'Amount', 'Vendor']]
# if len(df[c]):
dfc = df[c]
a = dfc.groupby([dfc['Date'].dt.year, dfc['Date'].dt.month]).Amount.sum()
# dfc.groupby(dfc['Date'].dt.month).Amount.sum()
# dfc.Amount.sum()
# cols = ['Date', 'Title', 'Vendor', 'Amount']
print(len(dfc))
dfc.iloc[0:200][cols]
# a

7


,Date,Title,Vendor,Amount
,,,,
636,2024-01-31,DOP. MC 557519******1718 PŁATNOŚĆ KARTĄ 12.00 PLN Goraco Polecam Wroclaw,nan,12.00
649,2024-01-25,For good cats life January,MISHCHENKO IULIIA UL BRZESKA 23M17 50-430 WROCŁAW ELIXIR 24-01-2024,-3372.00
658,2024-01-21,DOP. MC 557519******1718 PŁATNOŚĆ KARTĄ 16.17 PLN MULTICOOK WROCLAW,nan,16.17
665,2024-01-15,DOP. MC 557519******1718 PŁATNOŚĆ KARTĄ 14.00 PLN LOOPYS WORLD BIELANY WROCL,nan,14.00
666,2024-01-15,DOP. MC 557519******1718 PŁATNOŚĆ KARTĄ 9.00 PLN LOOPYS WORLD BIELANY WROCL,nan,9.00
667,2024-01-15,DOP. MC 557519******1718 PŁATNOŚĆ KARTĄ 10.00 PLN LOOPYS WORLD BIELANY WROCL,nan,10.00
668,2024-01-15,DOP. MC 557519******1718 PŁATNOŚĆ KARTĄ 56.80 PLN LOOPYS WORLD BIELANY WROCL,nan,56.80


In [ ]:
dfc.drop((dfc.Amount < 50))

KeyError: '[True, True, False, True, True, True, True, True, True, True, False, False, True, True, False, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True] not found in axis'